In [1]:
import sys
cpath = !pwd
sys.path.append(cpath[0][:-43]) 
sys.path.append(cpath[0][:-43]+'/algorithms')

In [2]:
from lionforests import LionForests
from algorithms.simpleSurrogate import GlobalSurrogateTree, LocalSurrogateTree
from algorithms.DefragTrees.defragTrees import DefragModel
from CHIRPS.structures import data_container
import CHIRPS.routines as rt
import CHIRPS.structures as strcts

from scipy import sparse

from datasets.dataset import Dataset
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import pandas as pd 
import numpy as np
np.seterr(invalid='ignore')
import warnings
warnings.filterwarnings("ignore")
import time

from sklearn.model_selection import train_test_split, LeaveOneOut
from sklearn.metrics import f1_score, precision_score

from anchor import anchor_tabular

segment = Dataset()

In [3]:
X, y, feature_names, class_names = segment.load_segment()

In [4]:
y = [int(i) for i in y]
y = np.array(y)
#one_h_percent = int(min(10*len(X)/100,100))
#print("Instances:",one_h_percent)

In [5]:
def measure(X_train, X_test, y_train, y_test, feature_names, class_names):
    parameters = [{
        'max_depth': [10],
        'max_features': [0.75],
        'bootstrap': [False],
        'min_samples_leaf' : [1],
        'n_estimators': [500]
    }]
    lf = LionForests(None, False, None, feature_names, class_names)
    lf.fit(X_train, y_train, params=parameters)    
    train = lf.utilizer.transform(X_train)
    test = lf.utilizer.transform(X_test)

    predictions = lf.model.predict(train)
    test_predictions = lf.model.predict(test)
    
    def techniques(model, train, y_train, predictions, test, feature_names, class_names, lf, task):

        #BaselineTechnique ==============================================================================
        gt = GlobalSurrogateTree(train, predictions, feature_names, task)
        lt = LocalSurrogateTree(train, predictions, feature_names, task, 150)

        #DefragTechnique ================================================================================
        Kmax = 10
        splitter = DefragModel.parseSLtrees(model) # parse sklearn tree ensembles into the array of (feature index, threshold)
        mdl = DefragModel(modeltype=task, maxitr=100, qitr=0, tol=1e-6, restart=20, verbose=0)
        mdl.fit(train, predictions, splitter, Kmax, fittype='FAB', featurename=feature_names)
        def def_cov(instances):
            ts = time.time()
            score, cover, coll = mdl.evaluate(instances[1:],lf.model.predict(instances[1:]))
            def_predictions = mdl.predict(instances[1:])
            rule = mdl.find_rule_length(instances[0])
            return rule, cover, def_predictions, time.time() - ts

        #Anchors =======================================================================================
        explainer = anchor_tabular.AnchorTabularExplainer(class_names, feature_names, train)
        def anchors_method(instance):
            exp = explainer.explain_instance(instance, lf.model.predict, threshold=0.95)
            anchors_dict = {}
            for i in exp.names():
                terms = i.split(' ')
                if len(terms) == 3:
                    anchors_dict[terms[0]] = [[terms[1],float(terms[2])]]
                else:
                    anchors_dict[terms[2]] = [[terms[3],float(terms[4])],[terms[1],float(terms[0])]]
            return anchors_dict, 0

        #CHIRPS =======================================================================================
        project_dir = cpath[0][:-43]+'/algorithms/CHIRPS'
        temp_y = np.array([int(i+1) for i in y_train])
        temp_frame = pd.DataFrame(np.hstack((train,temp_y.reshape(len(temp_y),1))),columns=feature_names+['class'])
        temp_frame['class']=temp_frame['class'].astype(int)
        mydata = data_container(
                data = temp_frame, class_col = 'class', var_names = feature_names,
                project_dir = project_dir, save_dir = 'segment', random_state=123)
        meta_data = mydata.get_meta()
        f_walker = strcts.classification_trees_walker(forest=model, meta_data=meta_data)
        f_walker.forest_walk(instances = test, labels = model.predict(test), forest_walk_async = True)

        explanations = strcts.CHIRPS_container(f_walker.path_detail,
                                        forest=model,
                                        sample_instances=sparse.csr_matrix(train), # any representative sample can be used
                                        sample_labels=predictions,
                                        meta_data=meta_data)
        chts = time.time()
        explanations.run_explanations(target_classes=model.predict(test), # we're explaining the prediction, not the true label!
                                explanation_async=False,
                                random_state=123,
                                which_trees='majority',
                                alpha_paths=0.0,
                                support_paths=0.1,
                                score_func=1,
                                precis_threshold=0.99,
                                disc_path_bins=4,
                                merging_bootstraps=20,
                                pruning_bootstraps=20,
                                delta=0.2,
                                weighting='kldiv')
        chte = (time.time() - chts)/len(test)
        def chirps_method(idx):
            chirps_dict = {}
            for i in explanations.explainers[idx].pruned_rule:
                if i[1]:
                    chirps_dict[i[0]] = [['<=',float(i[2])]]
                else:
                    chirps_dict[i[0]] = [['>',float(i[2])]]
            return chirps_dict, 0, chte

        #LionForests
        def lf_rule(instance):
            temp = lf.explain(instance)[5]
            rule = {}
            for key,value in temp.items():
                rule[key] = [['<=',value[1]],['>',value[0]]]
            return rule

        return {'gs':gt.rule,'ls':lt.rule,'an':anchors_method,'lf':lf_rule, 'df': def_cov, 'ch':chirps_method}
    interpretation = techniques(lf.model, train, y_train, predictions, test, feature_names, class_names, lf, 'classification')
    def rule_cov(instance, feature_names, rule):
        covered = True
        for k in range(len(instance)):
            feature = feature_names[k]
            if feature in rule.keys():
                if len(rule[feature]) == 2:
                    if instance[k] > rule[feature][0][1]: #<=
                        return 0
                    if instance[k] <= rule[feature][1][1]:#THIS <=
                        return 0
                elif rule[feature][0][0] == '>':
                    if instance[k] <= rule[feature][0][1]:
                        return 0
                else:
                    if instance[k] > rule[feature][0][1]: #<=
                        return 0
        return 1
    def rule_cov_LF(instance, feature_names, rule):
        covered = True
        for k in range(len(instance)):
            feature = feature_names[k]
            if feature in rule.keys():
                if len(rule[feature]) == 2:
                    if instance[k] > rule[feature][0][1]: #<=
                        return 0
                    if instance[k] < rule[feature][1][1]:#THIS <=
                        return 0
                elif rule[feature][0][0] == '>':
                    if instance[k] <= rule[feature][0][1]:
                        return 0
                else:
                    if instance[k] > rule[feature][0][1]: #<=
                        return 0
        return 1

    gt = GlobalSurrogateTree(train,lf.model.predict(train),feature_names,'c')
    lt = LocalSurrogateTree(train,lf.model.predict(train),feature_names,'c',150)

    rule_generator = interpretation
    full_coverage = {'gs':0, 'ls':0, 'an':0, 'lf':0, 'df':0, 'ch': 0}
    rule_length = {'gs':0, 'ls':0, 'an':0, 'lf':0, 'df':0, 'ch': 0}
    f_precision = {'gs':[], 'ls':[], 'an':[], 'lf':[], 'df':[], 'ch':[]}
    time_response = {'gs':[], 'ls':[], 'an':[], 'lf':[], 'df':[], 'ch':[]}

    x_train_temp = train
    x_test_temp = test

    y_train_temp = predictions
    y_test_temp = test_predictions
    
    x_train_temp_lf = lf.utilizer.inverse_transform(x_train_temp)
    x_test_temp_lf = lf.utilizer.inverse_transform(x_test_temp)    
    ktime = time.time()
    
    for test_ind in range(len(test)):
        if test_ind % 100 ==0:
            print(round(test_ind/(len(test))*100,2),'in:', time.time()-ktime)
            ktime = time.time()
        for name, method in rule_generator.items():
            if name == 'ch':
                rule, op, te = method(test_ind)
                coverage = 0
                precision = []
                co = 0
                for i in x_test_temp:
                    res = rule_cov(i, feature_names, rule)
                    coverage = coverage + res
                    if res == 1:
                        precision.append([y_test_temp[test_ind], y_test_temp[co]])
                    co = co + 1
                if len(precision) > 1:
                    precision = np.array(precision)
                    f_precision[name].append(precision_score(precision[:,:1],precision[:,1:],average='micro'))
                full_coverage[name] = full_coverage[name] + coverage/len(x_test_temp)
            elif name == 'df':
                #FIX RULE LENGTH!
                rule, cover, predictions, te = method(np.concatenate((x_test_temp[test_ind:test_ind+1],x_test_temp)))
                f_precision[name].append(precision_score(predictions,y_test_temp,average='micro'))
                full_coverage[name] = full_coverage[name] + cover
            elif name == 'lf':
                ts = time.time()
                rule = method(x_test_temp_lf[test_ind])
                te = time.time() - ts
                coverage = 0
                precision = []
                co = 0
                for i in x_test_temp_lf:
                    res = rule_cov_LF(i, feature_names, rule)
                    coverage = coverage + res
                    if res == 1:
                        precision.append([y_test_temp[test_ind], y_test_temp[co]])
                    co = co + 1
                if len(precision) > 1:
                    precision = np.array(precision)
                    f_precision[name].append(precision_score(precision[:,:1],precision[:,1:],average='micro'))
                full_coverage[name] = full_coverage[name] + coverage/len(x_test_temp_lf)
            else:
                ts = time.time()
                rule, prediction = method(x_test_temp[test_ind])
                te = time.time() - ts
                coverage = 0
                precision = []
                co = 0 
                for i in x_test_temp:
                    if name == 'anchors':
                        res = rule_cov(i, feature_names, rule)
                    else:    
                        res = rule_cov(i, feature_names, rule)
                    coverage = coverage + res
                    if res == 1 and name=='anchors':
                        precision.append([y_test_temp[test_ind], y_test_temp[co]])
                    elif res == 1:
                        precision.append([prediction, y_test_temp[co]])
                    co = co + 1
                if len(precision) > 1:
                    precision = np.array(precision)
                    f_precision[name].append(precision_score(precision[:,:1],precision[:,1:],average='micro'))
                full_coverage[name] = full_coverage[name] + coverage/len(x_test_temp)
            time_response[name].append(te)
            rule_length[name] = rule_length[name] + len(rule)
    return rule_generator, full_coverage, rule_length, f_precision, time_response

In [6]:
def print_current(total_results):
    full_coverage = {'gs':[], 'ls':[], 'an':[], 'lf':[], 'df':[], 'ch': []}
    rule_length = {'gs':[], 'ls':[], 'an':[], 'lf':[], 'df':[], 'ch': []}
    f_precision = {'gs':[], 'ls':[], 'an':[], 'lf':[], 'df':[], 'ch':[]}
    f_time = {'gs':[], 'ls':[], 'an':[], 'lf':[], 'df':[], 'ch':[]}

    for i in total_results:
        for name, method in i[0].items():
            full_coverage[name].append(i[1][name]/len(i[4][name]))
            rule_length[name].append(i[2][name]/len(i[4][name]))
            if len(i[3][name]) >= 1:
                f_precision[name].append(np.array(i[3][name]))                
            f_time[name].append(np.array(i[4][name]))
    for name, method in i[0].items():
        print(name,np.array(full_coverage[name]).mean(),',',
              np.array(rule_length[name]).mean(),',',
              np.array(f_precision[name]).mean(),',',
              np.array(f_time[name]).mean())

In [7]:
total_results = []
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=7)
results = measure(X_train, X_test, y_train, y_test, feature_names, class_names)
total_results.append(results)
total_results.append(results)
print_current(total_results)

len self.path_detail
462
Working on CHIRPS for instance 0 of 462
as_chirps for batch_idx 0
start mining for batch_idx 0 with support = 0.1
reduced 305 patterns out of 650 by numeric redundancy
found 345 patterns from 495 trees for batch_idx 0
start score sort for batch_idx 0 (345) patterns
start merge rule for batch_idx 0 (345) patterns
[('hue-mean', True, 0.18032), ('rawred-mean', True, 0.00324), ('region-centroid-row', True, 0.50471)]
0.8717948717948718 0.03818181323109765 0.1402048599810505 0.22354841900491282
merge complete for batch_idx 0 (345) patterns
start get explainer for batch_idx 0
as_chirps for batch_idx 1
start mining for batch_idx 1 with support = 0.1
found 148 patterns from 499 trees for batch_idx 1
start score sort for batch_idx 1 (148) patterns
start merge rule for batch_idx 1 (148) patterns
[('hue-mean', False, 0.14151), ('rawred-mean', False, 0.18144), ('region-centroid-row', True, 0.61229), ('intensity-mean', True, 0.58637), ('saturatoin-mean', True, 0.40267)]
0.95

[('hue-mean', True, 0.61909), ('region-centroid-row', False, 0.6098958477377892), ('exblue-mean', False, 0.23377), ('rawgreen-mean', False, 0.13949)]
0.9706959706959707 0.14322848637234414 0.40347850508096245 0.47563176895306847
merge complete for batch_idx 14 (41) patterns
start get explainer for batch_idx 14
Working on CHIRPS for instance 15 of 462
as_chirps for batch_idx 15
start mining for batch_idx 15 with support = 0.1
reduced 230 patterns out of 823 by numeric redundancy
found 593 patterns from 352 trees for batch_idx 15
start score sort for batch_idx 15 (593) patterns
start merge rule for batch_idx 15 (593) patterns
[('hue-mean', True, 0.19106), ('rawred-mean', True, 0.00616), ('region-centroid-row', True, 0.56598), ('region-centroid-col', True, 0.59665)]
0.8547008547008547 0.058892843139390506 0.13197983403403546 0.22715587312388974
merge complete for batch_idx 15 (593) patterns
start get explainer for batch_idx 15
as_chirps for batch_idx 16
start mining for batch_idx 16 with 

[('hue-mean', False, 0.4665788374841213)]
0.8590163934426229 0.15691124608150472 0.18295215986492488 0.27522255192878337
merge complete for batch_idx 29 (12) patterns
start get explainer for batch_idx 29
Working on CHIRPS for instance 30 of 462
as_chirps for batch_idx 30
start mining for batch_idx 30 with support = 0.1
found 12 patterns from 500 trees for batch_idx 30
start score sort for batch_idx 30 (12) patterns
start merge rule for batch_idx 30 (12) patterns
[('hue-mean', False, 0.4665788374841213)]
0.8590163934426229 0.15691124608150472 0.18295216277858498 0.27522255192878337
merge complete for batch_idx 30 (12) patterns
start get explainer for batch_idx 30
as_chirps for batch_idx 31
start mining for batch_idx 31 with support = 0.1
reduced 23 patterns out of 638 by numeric redundancy
found 615 patterns from 500 trees for batch_idx 31
start score sort for batch_idx 31 (615) patterns
start merge rule for batch_idx 31 (615) patterns
[('exgreen-mean', True, 0.47352), ('hue-mean', Fals

[('hue-mean', True, 0.61909), ('region-centroid-row', False, 0.6098958477377892), ('exblue-mean', False, 0.23377), ('rawgreen-mean', False, 0.13949)]
0.9706959706959707 0.14322848637234414 0.40347863426517394 0.47563176895306847
merge complete for batch_idx 45 (41) patterns
start get explainer for batch_idx 45
as_chirps for batch_idx 46
start mining for batch_idx 46 with support = 0.1
reduced 13 patterns out of 588 by numeric redundancy
found 575 patterns from 500 trees for batch_idx 46
start score sort for batch_idx 46 (575) patterns
start merge rule for batch_idx 46 (575) patterns
[('exgreen-mean', True, 0.47563), ('hue-mean', False, 0.20033), ('rawred-mean', True, 0.18124), ('exred-mean', False, 0.68687), ('vedge-mean', True, 0.09601)]
0.96875 0.13407483955183291 0.08492389929761178 0.11554281440857539
merge complete for batch_idx 46 (575) patterns
start get explainer for batch_idx 46
as_chirps for batch_idx 47
start mining for batch_idx 47 with support = 0.1
reduced 1 patterns out 

[('hue-mean', True, 0.61909), ('region-centroid-row', False, 0.6098958477377892), ('exblue-mean', False, 0.23377), ('rawgreen-mean', False, 0.13949)]
0.9706959706959707 0.14322848637234414 0.40347865199228605 0.47563176895306847
merge complete for batch_idx 60 (41) patterns
start get explainer for batch_idx 60
as_chirps for batch_idx 61
start mining for batch_idx 61 with support = 0.1
reduced 4 patterns out of 156 by numeric redundancy
found 152 patterns from 499 trees for batch_idx 61
start score sort for batch_idx 61 (152) patterns
start merge rule for batch_idx 61 (152) patterns
[('hue-mean', False, 0.14423), ('rawred-mean', False, 0.18109), ('region-centroid-row', True, 0.61458), ('intensity-mean', True, 0.58637), ('rawgreen-mean', False, 0.15705)]
0.9696969696969697 0.1206896551724138 0.16125886696458255 0.21391672371942483
merge complete for batch_idx 61 (152) patterns
start get explainer for batch_idx 61
as_chirps for batch_idx 62
start mining for batch_idx 62 with support = 0.1

...

[('hue-mean', True, 0.61909), ('region-centroid-row', False, 0.6098958477377892), ('exblue-mean', False, 0.23377), ('rawgreen-mean', False, 0.13949)]
0.9706959706959707 0.14322848637234414 0.4034785600165741 0.47563176895306847
merge complete for batch_idx 390 (41) patterns
start get explainer for batch_idx 390
as_chirps for batch_idx 391
start mining for batch_idx 391 with support = 0.1
found 12 patterns from 500 trees for batch_idx 391
start score sort for batch_idx 391 (12) patterns
start merge rule for batch_idx 391 (12) patterns
[('hue-mean', False, 0.4665788374841213)]
0.8590163934426229 0.15691124608150472 0.18295233653654766 0.27522255192878337
merge complete for batch_idx 391 (12) patterns
start get explainer for batch_idx 391
as_chirps for batch_idx 392
start mining for batch_idx 392 with support = 0.1
reduced 1 patterns out of 42 by numeric redundancy
found 41 patterns from 500 trees for batch_idx 392
start score sort for batch_idx 392 (41) patterns
start merge rule for batc

reduced 16 patterns out of 654 by numeric redundancy
found 638 patterns from 500 trees for batch_idx 406
start score sort for batch_idx 406 (638) patterns
start merge rule for batch_idx 406 (638) patterns
[('exgreen-mean', True, 0.48382), ('hue-mean', False, 0.20404), ('rawred-mean', True, 0.18124), ('exred-mean', False, 0.68677), ('vedge-mean', True, 0.09601)]
0.9682539682539683 0.13192102686826934 0.05659161229198689 0.07881959770477334
merge complete for batch_idx 406 (638) patterns
start get explainer for batch_idx 406
as_chirps for batch_idx 407
start mining for batch_idx 407 with support = 0.1
reduced 1 patterns out of 42 by numeric redundancy
found 41 patterns from 500 trees for batch_idx 407
start score sort for batch_idx 407 (41) patterns
start merge rule for batch_idx 407 (41) patterns
[('hue-mean', True, 0.61909), ('region-centroid-row', False, 0.6098958477377892), ('exblue-mean', False, 0.23377), ('rawgreen-mean', False, 0.13949)]
0.9706959706959707 0.14322848637234414 0.40

reduced 16 patterns out of 654 by numeric redundancy
found 638 patterns from 500 trees for batch_idx 421
start score sort for batch_idx 421 (638) patterns
start merge rule for batch_idx 421 (638) patterns
[('exgreen-mean', True, 0.48382), ('hue-mean', False, 0.20404), ('rawred-mean', True, 0.18124), ('exred-mean', False, 0.68677), ('vedge-mean', True, 0.09601)]
0.9682539682539683 0.13192102686826934 0.056592510956169 0.07882456804136717
merge complete for batch_idx 421 (638) patterns
start get explainer for batch_idx 421
as_chirps for batch_idx 422
start mining for batch_idx 422 with support = 0.1
reduced 8 patterns out of 652 by numeric redundancy
found 644 patterns from 500 trees for batch_idx 422
start score sort for batch_idx 422 (644) patterns
start merge rule for batch_idx 422 (644) patterns
[('exgreen-mean', True, 0.48382), ('hue-mean', False, 0.20396), ('rawred-mean', True, 0.18124), ('vedge-mean', True, 0.09601), ('exred-mean', False, 0.68701), ('rawgreen-mean', True, 0.54209)

as_chirps for batch_idx 436
start mining for batch_idx 436 with support = 0.1
reduced 16 patterns out of 805 by numeric redundancy
found 789 patterns from 500 trees for batch_idx 436
start score sort for batch_idx 436 (789) patterns
start merge rule for batch_idx 436 (789) patterns
[('exgreen-mean', True, 0.48382), ('hue-mean', False, 0.19884), ('rawred-mean', True, 0.18124), ('vedge-mean', True, 0.09601)]
0.9649805447470817 0.13452830958337866 0.041717621846981 0.06313691528359423
merge complete for batch_idx 436 (789) patterns
start get explainer for batch_idx 436
as_chirps for batch_idx 437
start mining for batch_idx 437 with support = 0.1
reduced 7 patterns out of 221 by numeric redundancy
found 214 patterns from 500 trees for batch_idx 437
start score sort for batch_idx 437 (214) patterns
start merge rule for batch_idx 437 (214) patterns
[('rawred-mean', True, 0.18051), ('region-centroid-col', False, 0.45152), ('hue-mean', False, 0.20034), ('intensity-mean', True, 0.02758)]
0.8541

[('hue-mean', True, 0.14123), ('region-centroid-row', True, 0.61196), ('intensity-mean', True, 0.58637)]
0.8756476683937824 0.09913929248262791 0.21297815207111093 0.3320312500000001
merge complete for batch_idx 450 (86) patterns
start get explainer for batch_idx 450
as_chirps for batch_idx 451
start mining for batch_idx 451 with support = 0.1
reduced 2 patterns out of 48 by numeric redundancy
found 46 patterns from 500 trees for batch_idx 451
start score sort for batch_idx 451 (46) patterns
start merge rule for batch_idx 451 (46) patterns
[('hue-mean', True, 0.61909), ('region-centroid-row', False, 0.6126), ('exblue-mean', False, 0.23377)]
0.9601449275362319 0.14456934648206202 0.2081848544620568 0.3429744355290165
merge complete for batch_idx 451 (46) patterns
start get explainer for batch_idx 451
as_chirps for batch_idx 452
start mining for batch_idx 452 with support = 0.1
reduced 13 patterns out of 588 by numeric redundancy
found 575 patterns from 500 trees for batch_idx 452
start 

In [16]:
#Me '0.75'
full_coverage = {'gs':[], 'ls':[], 'an':[], 'lf':[], 'df':[], 'ch': []}
rule_length = {'gs':[], 'ls':[], 'an':[], 'lf':[], 'df':[], 'ch': []}
f_precision = {'gs':[], 'ls':[], 'an':[], 'lf':[], 'df':[], 'ch':[]}

for i in total_results:
    for name, method in i[0].items():
        full_coverage[name].append(i[1][name]/one_h_percent)
        rule_length[name].append(i[2][name]/one_h_percent)
        f_precision[name].append(np.array(i[3][name]).mean())
for name, method in i[0].items():
    print(name,np.array(full_coverage[name]).mean(),',',np.array(full_coverage[name]).std(),
          np.array(rule_length[name]).mean(),',',np.array(rule_length[name]).std(),
          np.array(f_precision[name]).mean(),',',np.array(f_precision[name]).std())

gs 0.09282828282828283 , 0.006199914096386451 3.729 , 0.4088875150943105 0.9860838623403373 , 0.017836697542138076
ls 0.4437373737373739 , 0.014998010275134432 1.589 , 0.09771898484941399 0.3758100683772073 , 0.023161896099916396
an 0.0561111111111111 , 0.003669114579259564 4.886 , 0.19880643852752855 0.12840889109653503 , 0.024244090213287786
lf 0.07048484848484848 , 0.004890140919902875 8.764 , 0.4020994901762498 1.0 , 0.0
df 0.9869999999999998 , 0.013453624047074004 10.097999999999999 , 1.631685018623386 0.67 , 0.030331501776206277
ch 0.28078787878787875 , 0.008148930034556117 3.4490000000000003 , 0.1506286825275983 0.4310500112078371 , 0.00948842497186877
